# Pandas RAG 모델 구축

**[필요한 라이브러리 호출 및 API키 설정]**

In [1]:
from dotenv import load_dotenv; load_dotenv()  # dev에서만
import os
OPENAI_API = os.getenv("OPENAI_API")

In [2]:
import os
os.chdir(r'C:\Users\Hopedom\Documents\DS5-LangChain\Langchain-RAG\\')

## **[문서 로드/분할 및 벡터 임베딩]**

### [문서를 LangChain Document 객체로 로드]

</br>

$$\text{DirectoryLoader} \xrightarrow{\text{탐색 및 경로 전달}} \text{UnstructuredFileLoader} \xrightarrow{\text{파싱 및 텍스트 추출}} \text{Document 객체}$$


- `DirectoryLoader`는 `pandas/doc/source 디렉터리` 내부를 탐색하며, 발견된 모든 `rst` 파일 경로를 `UnstructuredFileLoader`에게 전달하는 역할 수행
- `UnstructuredFileLoader`는 `DirectoryLoader`가 찾은 개별 rst 파일 경로를 받아, 파일 내용을 읽고 rst 마크업을 어느 정도 제거하여 순수한 텍스트를 추출하는 실질적인 파싱 작업을 수행

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredFileLoader
from langchain_chroma import Chroma # 구 버전: from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from typing import List

# 📌 1. 로컬 .rst 파일 로드
# 로컬 Pandas 문서 소스 경로
PANDAS_DOC_PATH = "../pandas/doc/source"
print(f"로컬 경로 '{PANDAS_DOC_PATH}'에서 .rst 파일 로드 시작...")

# DirectoryLoader: 지정된 경로에서 .rst 파일을 찾고 UnstructuredFileLoader로 로드합니다.
# .rst 파일은 일반 텍스트 파일이므로 정확한 파싱을 위해 UnstructuredFileLoader를 사용합니다.
loader = DirectoryLoader(
    path=PANDAS_DOC_PATH,
    glob="**/*.rst",  # 재귀적으로 모든 .rst 파일 검색
    loader_cls=UnstructuredFileLoader,
    loader_kwargs={"autodetect_encoding": True},
    show_progress=True
)

# .rst 파일의 내용을 LangChain Document 객체로 로드
documents = loader.load()

print(f"로드된 Pandas 문서 객체 개수: {len(documents)}개")

로컬 경로 '../pandas/doc/source'에서 .rst 파일 로드 시작...


  9%|▉         | 19/211 [00:24<01:30,  2.12it/s][WARNING] Reference not found for 'np.r' at C:\Users\Hopedom\Documents\DS5-LangChain\Langchain-RAG\..\pandas\doc\source\getting_started\comparison\comparison_with_r.rst line 579 column 1

  9%|▉         | 20/211 [00:25<01:43,  1.84it/s][WARNING] Could not load include file includes/introduction.rst at C:\Users\Hopedom\Documents\DS5-LangChain\Langchain-RAG\..\pandas\doc\source\getting_started\comparison\comparison_with_sas.rst line 15 column 1
[WARNING] Could not load include file includes/copies.rst at C:\Users\Hopedom\Documents\DS5-LangChain\Langchain-RAG\..\pandas\doc\source\getting_started\comparison\comparison_with_sas.rst line 71 column 1
[WARNING] Could not load include file includes/construct_dataframe.rst at C:\Users\Hopedom\Documents\DS5-LangChain\Langchain-RAG\..\pandas\doc\source\getting_started\comparison\comparison_with_sas.rst line 94 column 1
[WARNING] Could not load include file includes/limit.rst at C:\Users\Hopedom\Docum

로드된 Pandas 문서 객체 개수: 211개


### [청크 분할 및 메타데이터 추가]

In [4]:
# PDF 파일 예시와 달리, 문서 원본이 rst이므로 chunk_overlap을 200으로 설정하여 문맥 보존 강화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200, 
    separators=["\n\n", "\n", " ", ""] # rst 마크업을 고려한 기본 분리자
)
chunks = text_splitter.split_documents(documents)

# RAG 멀티 도메인 필터링을 위해 'library': 'pandas' 메타데이터 추가
for chunk in chunks:
    # DirectoryLoader가 'source' 경로를 자동으로 추가해줍니다.
    chunk.metadata['library'] = 'pandas' 
    
print(f"분할된 최종 청크 개수: {len(chunks)}개")
print(f"첫 번째 청크의 메타데이터: {chunks[0].metadata}")

분할된 최종 청크 개수: 3449개
첫 번째 청크의 메타데이터: {'source': '..\\pandas\\doc\\source\\development\\community.rst', 'library': 'pandas'}


### [벡터 임베딩 및 ChromaDB 저장]

In [ ]:
#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
vectorstore = Chroma.from_documents(
    chunks, 
    OpenAIEmbeddings(model = 'text-embedding-3-small'),
    persist_directory='./chromadb/pandas_rst' 
)
retriever = vectorstore.as_retriever()

print("✅ Pandas RAG 데이터베이스 구축 완료.")
print(f"ChromaDB 저장 위치: './chromadb/pandas_rst'")


벡터 임베딩 및 ChromaDB 저장 시작 (OpenAI 'text-embedding-3-small' 사용)...
✅ Pandas RAG 데이터베이스 구축 완료.
ChromaDB 저장 위치: './chromadb/pandas_rst'


## **[프롬프트와 모델 선언]**

In [6]:
from langchain_core.prompts import load_prompt, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 로드 (LangChain Hub 또는 수동 정의)

try:
    # LangChain Hub에서 공식 RAG 프롬프트를 로드합니다.
    # rlm/rag-prompt 대신 'lc://prompts/rag-prompt/rag-prompt' 경로를 사용합니다.
    prompt = load_prompt("lc://prompts/rag-prompt/rag-prompt")
    
    # 로드된 프롬프트의 유형과 메시지 수 확인
    print("INFO: LangChain Hub 프롬프트가 'load_prompt'를 통해 성공적으로 로드되었습니다.")
    # print(f"프롬프트 유형: {type(prompt)}, 메시지 수: {len(prompt.messages)}")

except Exception as e:
    # load_prompt가 실패하거나 인터넷 연결 문제 등이 있을 경우를 대비한 대체 방법
    print(f"경고: LangChain Hub 프롬프트 로드에 실패했습니다. (오류: {e}) 프롬프트를 수동으로 정의합니다.")
    
    # RAG 프롬프트를 수동으로 정의 (rlm/rag-prompt의 일반적인 템플릿과 유사)
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an assistant for question-answering tasks. "
                "Use the following pieces of retrieved context to answer the question. "
                "If you don't know the answer, just say that you don't know. "
                "Use three sentences maximum and keep the answer concise.\n\n"
                "Context: {context}",
            ),
            ("human", "{question}"),
        ]
    )

경고: LangChain Hub 프롬프트 로드에 실패했습니다. (오류: Loading from the deprecated github-based Hub is no longer supported. Please use the new LangChain Hub at https://smith.langchain.com/hub instead.) 프롬프트를 수동으로 정의합니다.


In [7]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [8]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\nContext: {context}"), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]

In [9]:
# 언어 모델 (LLM) 선언

# 질문-답변 생성에 사용할 모델 선언
llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

print(f"✅ LLM 선언 완료: {llm.model_name}")

✅ LLM 선언 완료: gpt-5-nano


## **[Chain 구축]**

1. 질문을 받습니다.
2. retriever가 문서 조각(docs)을 검색합니다.
3. format_docs 함수가 docs를 단일 문자열 {context}로 만듭니다.
4. {context}와 {question}이 prompt 템플릿에 들어갑니다.
5. LLM이 답변을 생성하고, StrOutputParser가 이를 문자열로 변환합니다.

In [10]:
# Retriever로 검색한 유사 문서의 내용을 하나의 string으로 결합하는 함수 (Format Docs)
def format_docs(docs):
    """검색된 LangChain Document 객체들을 하나의 문자열 컨텍스트로 결합합니다."""
    return "\n\n".join(doc.page_content for doc in docs)

In [11]:
rag_chain = (
    {
        # context: retriever의 검색 결과를 format_docs 함수를 통해 문자열로 전달
        "context": retriever | format_docs, 
        # question: 원본 질문을 그대로 다음 단계로 전달
        "question": RunnablePassthrough()
    }
    | prompt  # 이전에 수동 정의된 prompt 객체 사용
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인(rag_chain) 구축 완료.")

✅ RAG 체인(rag_chain) 구축 완료.


In [12]:
rag_chain.get_graph().print_ascii()

             +---------------------------------+          
             | Parallel<context,question>Input |          
             +---------------------------------+          
                    ***                ***                
                 ***                      ***             
               **                            ***          
+----------------------+                        **        
| VectorStoreRetriever |                         *        
+----------------------+                         *        
            *                                    *        
            *                                    *        
            *                                    *        
    +-------------+                       +-------------+ 
    | format_docs |                       | Passthrough | 
    +-------------+*                      +-------------+ 
                    ***                ***                
                       ***          ***                 

## [RAG 질의 테스트]

In [13]:
question = "Pandas에서 누락된 값(Missing Values)을 확인하는 가장 일반적인 메서드는 무엇인가요?"
print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas에서 누락된 값(Missing Values)을 확인하는 가장 일반적인 메서드는 무엇인가요?

[답변]: 가장 일반적으로 사용하는 방법은 isna() 또는 isnull()로 누락 값을 확인하는 것입니다. 이 함수들은 DataFrame이나 Series에서 누락 값 위치를 True로 표시하는 불리언 시퀀스를 반환합니다.


In [14]:
question = "Pandas에서 datetime 형식의 열을 처리하는 방법은 무엇인가요?"
print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas에서 datetime 형식의 열을 처리하는 방법은 무엇인가요?

[답변]: - 먼저 문자열인 열이라면 pd.to_datetime(열)으로 datetime64[ns, UTC]의 pandas.Timestamp 객체로 변환합니다. 
- 파일을 읽을 때는 parse_dates=[열]를 사용하면 자동으로 Timestamp로 변환됩니다. 
- 변환된 열은 df['datetime'].dt.year, df['datetime'].dt.dayofweek 등 dt 접근자로 연도, 요일 등 다양한 datetime 연산이 가능합니다.


In [36]:
question = "Pandas DataFrame에서 결측값을 처리하고, Scikit-learn Pipeline으로 StandardScaler와 LogisticRegression을 적용한 후, Stratified K-Fold 교차검증까지 수행하는 전체 예제 코드를 보여줘"

print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas DataFrame에서 결측값을 처리하고, Scikit-learn Pipeline으로 StandardScaler와 LogisticRegression을 적용한 후, Stratified K-Fold 교차검증까지 수행하는 전체 예제 코드를 보여줘

[답변]: 다음 예제는 Pandas DataFrame에서 결측값을 처리하고, Pipeline으로 StandardScaler와 LogisticRegression을 적용한 뒤 StratifiedKFold로 교차검증을 수행하는 전체 코드입니다.

```python
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score

# 데이터 생성 및 결측값 추가
rng = np.random.default_rng(42)
n_samples, n_features = 200, 4
X = rng.normal(size=(n_samples, n_features))
coefs = np.array([0.5, -0.3, 0.8, -0.2])
linear = X @ coefs
prob = 1 / (1 + np.exp(-linear))
y = (prob > 0.5).astype(int)

# 15%의 결측값 도입
mask = rng.random(size=X.shape) < 0.15
X[mask] = np.nan

df = pd.DataFrame(X, columns=[f'f{i+1}' for i in range(n_features)])
df['target'] = y

X_df = df.

In [ ]:
"Pandas DataFrame에서 결측값을 처리하고, Scikit-learn Pipeline으로 StandardScaler와 LogisticRegression을 적용한 후, Stratified K-Fold 교차검증까지 수행하는 전체 예제 코드를 보여줘"

question_examples = [
    "DataFrame에서 누락된 값(NaN)을 제거하는 예제를 보여줘",
    "Scikit-learn Pipeline에서 StandardScaler와 LogisticRegression을 사용하는 예제 코드 작성",
]

for q in question_examples:
    print("\n==============================")
    print(f"질문: {q}")
    response = rag_chain.invoke(q)
    print(f"답변:\n{response}")


질문: DataFrame에서 누락된 값(NaN)을 제거하는 예제를 보여줘
답변:
DataFrame에서 NaN이 있는 행을 제거하려면 dropna를 사용하면 됩니다. 예시:

import pandas as pd
import numpy as np

df = pd.DataFrame({"A":[1, np.nan, 3], "B":[4, 5, np.nan]})
df_clean = df.dropna()
print(df_clean)

질문: Scikit-learn Pipeline에서 StandardScaler와 LogisticRegression을 사용하는 예제 코드 작성
답변:
Scikit-learn의 Pipeline에서 StandardScaler와 LogisticRegression을 사용하는 간단한 예제 코드입니다.

```python
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000, random_state=42, solver='lbfgs', multi_class='multinomial'))
])

pipe.fit(X_train, y_train)
score = pipe.score(X_test, 

# scikit-learn 문서를 db에 추가 

### 🧭 [Scikit-learn 문서를 LangChain Document 객체로 로드]

</br>
(기존)  

$$
\text{DirectoryLoader} 
\xrightarrow{\text{경로 탐색 및 패턴 매칭 (``*.rst``, ``**/*.rst``)}} 
\text{UnstructuredRSTLoader} 
\xrightarrow{\text{Sphinx 기반 문서의 마크업 제거 및 텍스트 추출}} 
\text{Document 객체}
$$

$$
\text{DirectoryLoader} 
\xrightarrow{\text{경로 탐색 및 패턴 매칭 (``*.md``, ``**/*.md``)}} 
\text{UnstructuredFileLoader} 
\xrightarrow{\text{Markdown 문서의 본문 파싱 및 텍스트 정제}} 
\text{Document 객체}
$$

---

- `DirectoryLoader`는 **Scikit-learn 공식 문서 저장 경로 (`scikit-learn/doc/`)** 내부를 재귀적으로 탐색하며, `.rst` 및 `.md` 파일을 모두 찾아 각 경로를 개별 파일 로더로 전달하는 역할 수행

- `.rst` 파일의 경우,  
  `UnstructuredRSTLoader`를 활용하여 Sphinx 스타일 문법(`.. include::`, `:class:`, `:ref:` 등)을 인식해 가능한 한 순수 텍스트를 정제하여 `Document` 객체로 변환하였음

- `.md` 파일의 경우,  
  `UnstructuredFileLoader`를 활용하여 일반 마크다운 구문(`##`, `-`, 코드 블록 등)을 정제하여 텍스트 기반의 `Document` 객체로 변환하였음

- 이렇게 생성된 `Document` 객체는 이후 `RecursiveCharacterTextSplitter``를 활용하여 RAG 시스템에 적합한 형태(청크 단위)로 가공하였음


</br>
(최종버전)

$$
\text{DirectoryLoader} 
\xrightarrow{\text{경로 탐색 및 패턴 매칭 }} 
\text{UnstructuredFileLoader} 
\xrightarrow{\text{Markdown 문서의 본문 파싱 및 텍스트 정제}} 
\text{Document 객체}
$$

- `.rst` 파일에 사용한 `UnstructuredRSTLoader`의 경우, Sphinx 스타일의 문법을 인식하였음에도 일부 텍스트 정제에 실패했을 경우, 해당 문서를 로드하지 못하는 단점을 확인하였음
- 이에, `.rest` 파일 또한 `UnstructuredRSTLoader`를 활용하여 `Document` 객체로 변환하였음

In [34]:
# 1. Scikit-learn 문서 로드 (DirectoryLoader 사용)

SKLEARN_DOC_PATH = r"C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc" 
print(f"🔄 Scikit-learn 문서 로드 시작 ({SKLEARN_DOC_PATH} 폴더)...")

# 로드된 문서들을 저장할 임시 리스트
sklearn_docs_list = []

# --- 1-1. .rst 파일 로드 (.rst 파일만 해당) ---
print("  - .rst 파일 로드 시작")
# 최상위 파일 (glob="*") + 하위 폴더 파일 (glob="**/*") 모두 로드
for pattern in ["*.rst", "**/*.rst"]:
    loader = DirectoryLoader(
        path=SKLEARN_DOC_PATH, 
        glob=pattern, # .rst 확장자만 사용
        loader_cls=UnstructuredFileLoader,
        loader_kwargs={"autodetect_encoding": True}
    )
    sklearn_docs_list.extend(loader.load())
    print(f"    > 패턴 '{pattern}' 로드 후 현재 문서 수: {len(sklearn_docs_list)}개")

# --- 1-2. .md 파일 로드 (.md 파일만 해당) ---
print("  - .md 파일 로드 시작")
# 최상위 파일 (glob="*") + 하위 폴더 파일 (glob="**/*") 모두 로드
for pattern in ["*.md", "**/*.md"]:
    loader = DirectoryLoader(
        path=SKLEARN_DOC_PATH, 
        glob=pattern, # .md 확장자만 사용
        loader_cls=UnstructuredFileLoader,
        loader_kwargs={"autodetect_encoding": True}
    )
    sklearn_docs_list.extend(loader.load())
    print(f"    > 패턴 '{pattern}' 로드 후 현재 문서 수: {len(sklearn_docs_list)}개")


# 최종 결과 합산
sklearn_docs = sklearn_docs_list
print(f"✅ Scikit-learn 문서 로드 완료. 총 문서 수: {len(sklearn_docs)}개")

🔄 Scikit-learn 문서 로드 시작 (C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc 폴더)...
  - .rst 파일 로드 시작


[WARNING] Could not load include file maintainers.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 52 column 1
[WARNING] Could not load include file documentation_team.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 70 column 1
[WARNING] Could not load include file contributor_experience_team.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 79 column 1
[WARNING] Could not load include file communication_team.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 87 column 1
[WARNING] Could not load include file maintainers_emeritus.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 99 column 1
[WARNING] Could not load include file communication_team_emeritus.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 107 column 1
[WARNING] Could not load include file contributor_experience_team_emeritus.rst at C:\Users\H

    > 패턴 '*.rst' 로드 후 현재 문서 수: 33개


[WARNING] Could not load include file maintainers.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 52 column 1
[WARNING] Could not load include file documentation_team.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 70 column 1
[WARNING] Could not load include file contributor_experience_team.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 79 column 1
[WARNING] Could not load include file communication_team.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 87 column 1
[WARNING] Could not load include file maintainers_emeritus.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 99 column 1
[WARNING] Could not load include file communication_team_emeritus.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 107 column 1
[WARNING] Could not load include file contributor_experience_team_emeritus.rst at C:\Users\H

    > 패턴 '**/*.rst' 로드 후 현재 문서 수: 222개
  - .md 파일 로드 시작
    > 패턴 '*.md' 로드 후 현재 문서 수: 223개
    > 패턴 '**/*.md' 로드 후 현재 문서 수: 226개
✅ Scikit-learn 문서 로드 완료. 총 문서 수: 226개


### [청크 분할 및 메타데이터 추가]

In [40]:
# 청크 분할 (Chunking)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200, 
    separators=["\n\n", "\n", " ", ""] # rst 마크업을 고려한 기본 분리자
)
chunks = text_splitter.split_documents(sklearn_docs) # 로드된 sklearn_docs 사용

# RAG 멀티 도메인 필터링을 위해 'library': 'scikit-learn' 메타데이터 추가
for chunk in chunks:
    # 1. 라이브러리 구분 메타데이터 추가
    chunk.metadata['library'] = 'scikit-learn'

print(f"분할된 최종 청크 개수: {len(chunks)}개")
print(f"첫 번째 청크의 메타데이터: {chunks[0].metadata}")

분할된 최종 청크 개수: 3027개
첫 번째 청크의 메타데이터: {'source': 'C:\\Users\\Hopedom\\Documents\\DS5-LangChain\\scikit-learn\\doc\\about.rst', 'library': 'scikit-learn'}


### [벡터 임베딩 및 ChromaDB 저장]

In [ ]:
vectorstore = Chroma(
    persist_directory='./chromadb/pandas_rst' ,
    embedding_function=OpenAIEmbeddings(model='text-embedding-3-small')
)

initial_count = vectorstore._collection.count()
print(f"✅ 통합 전 DB 문서 수 (Pandas): {initial_count}개")

# Scikit-learn 청크 추가
vectorstore.add_documents(chunks)

# 3. DB 영구 저장 및 최종 확인
final_count = vectorstore._collection.count()
print(f"🎉 통합 완료! 최종 DB 문서 수: {final_count}개")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22108\2061794180.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


✅ 통합 전 DB 문서 수 (Pandas): 13796개
🎉 통합 완료! 최종 DB 문서 수: 16823개


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22108\2061794180.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [43]:
retriever = vectorstore.as_retriever()
print("✅ 통합 RAG 데이터베이스 구축 완료.")

✅ 통합 RAG 데이터베이스 구축 완료.


## **[Chain 구축]**

1. 질문을 받습니다.
2. retriever가 문서 조각(docs)을 검색합니다.
3. format_docs 함수가 docs를 단일 문자열 {context}로 만듭니다.
4. {context}와 {question}이 prompt 템플릿에 들어갑니다.
5. LLM이 답변을 생성하고, StrOutputParser가 이를 문자열로 변환합니다.

In [44]:
rag_chain = (
    {
        # context: retriever의 검색 결과를 format_docs 함수를 통해 문자열로 전달
        "context": retriever | format_docs, 
        # question: 원본 질문을 그대로 다음 단계로 전달
        "question": RunnablePassthrough()
    }
    | prompt  # 이전에 수동 정의된 prompt 객체 사용
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain 재구축 완료.")


✅ RAG Chain 재구축 완료.


In [45]:
rag_chain.get_graph().print_ascii()

             +---------------------------------+          
             | Parallel<context,question>Input |          
             +---------------------------------+          
                    ***                ***                
                 ***                      ***             
               **                            ***          
+----------------------+                        **        
| VectorStoreRetriever |                         *        
+----------------------+                         *        
            *                                    *        
            *                                    *        
            *                                    *        
    +-------------+                       +-------------+ 
    | format_docs |                       | Passthrough | 
    +-------------+*                      +-------------+ 
                    ***                ***                
                       ***          ***                 

## [RAG 질의 테스트]

In [47]:
question = "Pandas에서 누락된 값(Missing Values)을 확인하는 가장 일반적인 메서드는 무엇인가요?"
print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas에서 누락된 값(Missing Values)을 확인하는 가장 일반적인 메서드는 무엇인가요?

[답변]: 가장 일반적인 방법은 isna() 또는 isnull()를 사용하는 것입니다. 두 메서드는 동일하게 작동하며 누락값 위치에서 True를 반환합니다. 예를 들어 df['col'].isna()로 누락값 여부를 확인하거나 df.isna().sum()으로 누락값의 개수를 구할 수 있습니다.


In [48]:
question = "Scikit-learn의 Grid Search를 사용하여 하이퍼파라미터 튜닝하는 방법을 알려주세요."
print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Scikit-learn의 Grid Search를 사용하여 하이퍼파라미터 튜닝하는 방법을 알려주세요.

[답변]: GridSearchCV는 파라미터 그리드(param_grid)에 대해 모든 조합을 탐색하여 최적의 하이퍼파라미터를 찾는 scikit-learn 도구입니다. 사용 시 estimator(예: SVC), param_grid, cv(교차 검증 수), scoring 등을 전달하고 fit(X, y)를 실행하면 best_params_, best_estimator_, best_score_를 얻을 수 있습니다. 예시: grid = GridSearchCV(SVC(), {'C': [1, 10, 100], 'kernel': ['linear', 'rbf']}, cv=5, scoring='accuracy'); grid.fit(X, y); best_params_ = grid.best_params_.


In [51]:
question = """
Pandas 데이터프레임(df)에 문자열 범주형 특성(object dtype)과 수치형 특성(float64 dtype)이 혼합되어 있으며, 수치형 특성에는 결측치(NaN)가 포함된 이진 분류 문제를 해결해야 합니다.

Scikit-learn의 모범 사례에 따라 Pipeline과 ColumnTransformer를 사용하여 이 데이터를 전처리하고, 이어서 RandomForestClassifier의 최적 하이퍼파라미터를 찾고자 합니다.

아래 요구사항에 따라 응답해 주십시오.

1. **전처리 파이프라인 구성:** ColumnTransformer 내에서 범주형 특성(categorical_features)과 수치형 특성(numerical_features) 각각에 적용해야 할 가장 적절한 Scikit-learn 변환기(Transformer) 이름을 제시하고, 그 변환기를 선택한 이유를 간결하게 설명하십시오.
2. **하이퍼파라미터 튜닝 전략:** 구축한 파이프라인의 최종 추정기(RandomForestClassifier)에 대해 GridSearchCV를 사용하여 탐색할 핵심 하이퍼파라미터 2가지를 제시하고, 파이프라인 내에서 해당 하이퍼파라미터를 지정할 때 사용해야 하는 정확한 매개변수 명명법(파이프라인_단계_이름__하이퍼파라미터)의 예시를 제시하십시오.
"""

print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: 
Pandas 데이터프레임(df)에 문자열 범주형 특성(object dtype)과 수치형 특성(float64 dtype)이 혼합되어 있으며, 수치형 특성에는 결측치(NaN)가 포함된 이진 분류 문제를 해결해야 합니다.

Scikit-learn의 모범 사례에 따라 Pipeline과 ColumnTransformer를 사용하여 이 데이터를 전처리하고, 이어서 RandomForestClassifier의 최적 하이퍼파라미터를 찾고자 합니다.

아래 요구사항에 따라 응답해 주십시오.

1. **전처리 파이프라인 구성:** ColumnTransformer 내에서 범주형 특성(categorical_features)과 수치형 특성(numerical_features) 각각에 적용해야 할 가장 적절한 Scikit-learn 변환기(Transformer) 이름을 제시하고, 그 변환기를 선택한 이유를 간결하게 설명하십시오.
2. **하이퍼파라미터 튜닝 전략:** 구축한 파이프라인의 최종 추정기(RandomForestClassifier)에 대해 GridSearchCV를 사용하여 탐색할 핵심 하이퍼파라미터 2가지를 제시하고, 파이프라인 내에서 해당 하이퍼파라미터를 지정할 때 사용해야 하는 정확한 매개변수 명명법(파이프라인_단계_이름__하이퍼파라미터)의 예시를 제시하십시오.


[답변]: 1) ColumnTransformer의 cat 트랜스포머에는 SimpleImputer(strategy='most_frequent')와 OneHotEncoder(handle_unknown='ignore')를 차례로 적용하고, numerical 트랜스포머에는 SimpleImputer(strategy='median')를 적용하는 파이프라인을 매핑하는 것이 적합합니다. 2) 최적 하이퍼파라미터를 찾기 위해 RandomForestClassifier의 n_estimators와 max_depth를 GridSearchCV로 탐색하는 것이 일반적입니다. 3) 파이프라인 내 매개변수명은 'rf__n_estim

In [50]:
question = "Pandas DataFrame에서 결측값을 처리하고, Scikit-learn Pipeline으로 StandardScaler와 LogisticRegression을 적용한 후, Stratified K-Fold 교차검증까지 수행하는 전체 예제 코드를 보여줘"

print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas DataFrame에서 결측값을 처리하고, Scikit-learn Pipeline으로 StandardScaler와 LogisticRegression을 적용한 후, Stratified K-Fold 교차검증까지 수행하는 전체 예제 코드를 보여줘

[답변]: 다음은 pandas DataFrame에서 결측치를 처리하고, Pipeline으로 StandardScaler와 LogisticRegression를 적용한 뒤 StratifiedKFold 교차검증을 수행하는 전체 예제 코드입니다.

```python
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 샘플 데이터 생성 및 결측값 주입
np.random.seed(0)
df = pd.DataFrame({
    'feature1': np.random.randn(200),
    'feature2': np.random.randn(200),
    'feature3': np.random.randn(200),
})
for col in df.columns:
    df.loc[df.sample(frac=0.1).index, col] = np.nan

# 타깃 생성
df['target'] = (df['feature1'] + df['feature2'] > 0).astype(int)

X = df.drop(columns=['target'])
y = df['target']

# 파이프라인 구성: 결측값 임퓨테이션 > 표준화 > 로지스틱 회귀
pipe

## **[프롬프트와 모델 재선언]**

In [57]:
from langchain_core.prompts import ChatPromptTemplate

# -----------------------------------------------------------
# 수정된 System Instruction (설명 + 코드 제공 지침 포함)
# -----------------------------------------------------------
NEW_SYSTEM_INSTRUCTION_BALANCED = (
    "You are a professional Python code generation and technical documentation assistant. "
    "Use the following retrieved context to answer the question. "
    
    # 1. 자연어 답변 지침: 질문에 대해 먼저 간결하고 명확한 **자연어 답변**을 제공하십시오.
    "Start your response with a concise and clear natural language explanation, keeping the answer focused on the question."
    
    # 2. 코드 제공 지침: 코드가 요구되면, 자연어 답변 뒤에 **예시 코드**를 추가하십시오.
    "If the user's question explicitly asks for code (e.g., 'show code', 'example', 'how to implement'), "
    "or if providing code is helpful, include the code as a single, runnable Python code block (```python ... ```) "
    "after the natural language explanation. "
    
    "If you don't know the answer, just say that you don't know.\n\n"
    "Context: {context}"
)
# -----------------------------------------------------------

# 프롬프트 수동 정의 (NEW_SYSTEM_INSTRUCTION_BALANCED 사용)
print(f"INFO: 프롬프트를 수동으로 정의합니다. (설명 및 코드 균형 지침 포함)")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            NEW_SYSTEM_INSTRUCTION_BALANCED, # 균형 잡힌 시스템 지침 적용
        ),
        ("human", "{question}"),
    ]
)

# -----------------------------------------------------------
# Chain 재구축
# -----------------------------------------------------------
# 전제: retriever와 llm 변수는 이미 메모리에 정의되어 있어야 합니다.
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt # 수정된 프롬프트 사용
    | llm
    | StrOutputParser()
)
print("✅ RAG Chain이 설명 및 코드 균형 지침을 사용하여 재구축되었습니다.")

INFO: 프롬프트를 수동으로 정의합니다. (설명 및 코드 균형 지침 포함)
✅ RAG Chain이 설명 및 코드 균형 지침을 사용하여 재구축되었습니다.


### [RAG 질의 테스트]

In [58]:
question = "Pandas에서 누락된 값(Missing Values)을 확인하는 가장 일반적인 메서드는 무엇인가요?"
print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas에서 누락된 값(Missing Values)을 확인하는 가장 일반적인 메서드는 무엇인가요?

[답변]: 가장 일반적으로 사용하는 방법은 isna() 또는 isnull() 메서드로 누락값 여부를 불리언 마스크로 확인하는 것입니다. 두 메서드는 서로 같은 기능을 수행하는 동의어(alias)이며, notna()/notnull()은 반대 값을 제공합니다.

- 예: s.isna() 또는 s.isnull()은 각 원소가 누락인지 여부를 True/False로 반환
- 데이터프레임 전체에서 누락 여부 확인: df.isna()
- 열별 누락 개수: df.isna().sum()
- 특정 축 기준 누락 여부 확인: df.isna().any(axis=0) (열 기준), df.isna().any(axis=1) (행 기준)

```python
import pandas as pd
import numpy as np

# 예시 시리즈
s = pd.Series([1, np.nan, 3])
print("Series 누락 여부:\n", s.isna())

# 예시 데이터프레임
df = pd.DataFrame({"a": [1, np.nan, 3], "b": [np.nan, 5, 6]})
print("DataFrame 누락 여부:\n", df.isna())

print("컬럼별 누락 개수:\n", df.isna().sum())

print("행마다 하나라도 누락이 있나?:\n", df.isna().any(axis=1))
```

참고: isnull()도 동일하게 동작하므로 필요에 따라 취향대로 사용하면 됩니다.


In [59]:
question = "Scikit-learn의 Grid Search를 사용하여 하이퍼파라미터 튜닝하는 방법을 알려주세요."
print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Scikit-learn의 Grid Search를 사용하여 하이퍼파라미터 튜닝하는 방법을 알려주세요.

[답변]: Grid Search는 주어진 파라미터 그리드(param_grid)의 모든 조합을 교차 검증(cv)과 함께 평가해 최적의 하이퍼파라미터를 찾는 방법입니다. 파이프라인과 함께 사용하면 전처리 단계까지 포함한 최적의 파라미터를 찾을 수 있습니다. 파라미터 이름은 파이프라인의 각 스텝 이름으로 접두사(. 예: 'svc__C')를 붙여 지정합니다.

주요 포인트
- param_grid: 탐색할 파라미터 값의 사전(dict) 또는 사전의 리스트입니다. 리스트 형태로 여러 그리드를 합치는 방식으로 각 그리드의 조합을 탐색할 수 있습니다.
- GridSearchCV(estimator, param_grid, cv, scoring, n_jobs, refit): estimator은 파이프라인이나 모델, cv는 교차 검증 폴드 수, scoring은 평가 지표, n_jobs는 병렬 실행 수입니다.
- 결과 확인: best_params_, best_score_, cv_results_, best_estimator_ 등을 통해 최적 파라미터와 성능을 확인합니다.
- 파이프라인 사용 권장: 전처리와 모델을 함께 최적화할 수 있습니다.
- 다중 메트릭이나 사용자 정의 점수도 가능하며, refit 인자를 통해 최적 파라미터로 재학습한 최종 모델을 사용할 수 있습니다.

예시 코드
- Iris 데이터셋에 대해 SVC를 사용하고, 커널에 따라 다른 하이퍼파라미터를 탐색하는 예제입니다. 파이프라인에 표준화(StandardScaler)를 포함해 SVM의 성능을 안정화합니다.

```python
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pre

In [62]:
question = """
Pandas 데이터프레임(df)에 문자열 범주형 특성(object dtype)과 수치형 특성(float64 dtype)이 혼합되어 있으며, 수치형 특성에는 결측치(NaN)가 포함된 이진 분류 문제를 해결해야 합니다.

Scikit-learn의 모범 사례에 따라 Pipeline과 ColumnTransformer를 사용하여 이 데이터를 전처리하고, 이어서 RandomForestClassifier의 최적 하이퍼파라미터를 찾고자 합니다.

아래 요구사항에 따라 응답해 주십시오.

1. **전처리 파이프라인 구성:** ColumnTransformer 내에서 범주형 특성(categorical_features)과 수치형 특성(numerical_features) 각각에 적용해야 할 가장 적절한 Scikit-learn 변환기(Transformer) 이름을 제시하고, 그 변환기를 선택한 이유를 간결하게 설명하십시오.
2. **하이퍼파라미터 튜닝 전략:** 구축한 파이프라인의 최종 추정기(RandomForestClassifier)에 대해 GridSearchCV를 사용하여 탐색할 핵심 하이퍼파라미터 2가지를 제시하고, 파이프라인 내에서 해당 하이퍼파라미터를 지정할 때 사용해야 하는 정확한 매개변수 명명법(파이프라인_단계_이름__하이퍼파라미터)의 예시를 제시하십시오.
"""

print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: 
Pandas 데이터프레임(df)에 문자열 범주형 특성(object dtype)과 수치형 특성(float64 dtype)이 혼합되어 있으며, 수치형 특성에는 결측치(NaN)가 포함된 이진 분류 문제를 해결해야 합니다.

Scikit-learn의 모범 사례에 따라 Pipeline과 ColumnTransformer를 사용하여 이 데이터를 전처리하고, 이어서 RandomForestClassifier의 최적 하이퍼파라미터를 찾고자 합니다.

아래 요구사항에 따라 응답해 주십시오.

1. **전처리 파이프라인 구성:** ColumnTransformer 내에서 범주형 특성(categorical_features)과 수치형 특성(numerical_features) 각각에 적용해야 할 가장 적절한 Scikit-learn 변환기(Transformer) 이름을 제시하고, 그 변환기를 선택한 이유를 간결하게 설명하십시오.
2. **하이퍼파라미터 튜닝 전략:** 구축한 파이프라인의 최종 추정기(RandomForestClassifier)에 대해 GridSearchCV를 사용하여 탐색할 핵심 하이퍼파라미터 2가지를 제시하고, 파이프라인 내에서 해당 하이퍼파라미터를 지정할 때 사용해야 하는 정확한 매개변수 명명법(파이프라인_단계_이름__하이퍼파라미터)의 예시를 제시하십시오.


[답변]: 아래 방식은 Pandas DataFrame의 문자열 범주형 특성과 수치형 특성을 함께 다루면서, NaN이 있는 이진 분류 문제에 대해 Scikit-learn의 파이프라인과 ColumnTransformer를 활용한 표준적인 방법입니다. 범주형 특성은 원-핫 인코딩으로 변환하고, 수치형 특성은 평균으로 결측치를 대체합니다. 그 다음 RandomForestClassifier를 사용해 하이퍼파라미터를 GridSearchCV로 탐색합니다.

1) 전처리 파이프라인 구성 제안
- 범주형(features with object dtype) 처리기 이름: 'categorical'
  - 선택 이유: Simp

In [61]:
question = "Pandas DataFrame에서 결측값을 처리하고, Scikit-learn Pipeline으로 StandardScaler와 LogisticRegression을 적용한 후, Stratified K-Fold 교차검증까지 수행하는 전체 예제 코드를 보여줘"

print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas DataFrame에서 결측값을 처리하고, Scikit-learn Pipeline으로 StandardScaler와 LogisticRegression을 적용한 후, Stratified K-Fold 교차검증까지 수행하는 전체 예제 코드를 보여줘

[답변]: 다음은 Pandas DataFrame에서 결측값을 처리하고, Scikit-learn Pipeline으로 StandardScaler와 LogisticRegression을 적용한 뒤, Stratified K-Fold 교차검증까지 수행하는 완전한 예제 코드입니다. 결측값은 평균으로 대체(imputation)하고, 분류기를 파이프라인으로 구성하여 교차검증에 적용합니다.

```python
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 시드 고정 for 재현성
np.random.seed(0)

# 예제 데이터 생성: 150개 샘플, 4개 특성
n_samples = 150
X = np.random.randn(n_samples, 4)

# 간단한 선형 결합으로 이진 타깃 생성
coef = np.array([0.5, -1.2, 0.8, 0.3])
logit = X @ coef
prob = 1 / (1 + np.exp(-logit))
y = (prob > 0.5).astype(int)

# 10%의 값에 NaN 추가 (결측값 시뮬레이션)
mask = np.random.rand(n_samples, 4) < 0.1
X[mask] = np.na

In [ ]:
question = "Pandas에서 누락된 값(Missing Values)을 확인하는 가장 일반적인 메서드는 무엇인가요?"
print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas에서 누락된 값(Missing Values)을 확인하는 가장 일반적인 메서드는 무엇인가요?

[답변]: 가장 일반적인 방법은 isna() 또는 isnull()를 사용하는 것입니다. 두 메서드는 동일하게 작동하며 누락값 위치에서 True를 반환합니다. 예를 들어 df['col'].isna()로 누락값 여부를 확인하거나 df.isna().sum()으로 누락값의 개수를 구할 수 있습니다.


In [ ]:
question = "Scikit-learn의 Grid Search를 사용하여 하이퍼파라미터 튜닝하는 방법을 알려주세요."
print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Scikit-learn의 Grid Search를 사용하여 하이퍼파라미터 튜닝하는 방법을 알려주세요.

[답변]: GridSearchCV는 파라미터 그리드(param_grid)에 대해 모든 조합을 탐색하여 최적의 하이퍼파라미터를 찾는 scikit-learn 도구입니다. 사용 시 estimator(예: SVC), param_grid, cv(교차 검증 수), scoring 등을 전달하고 fit(X, y)를 실행하면 best_params_, best_estimator_, best_score_를 얻을 수 있습니다. 예시: grid = GridSearchCV(SVC(), {'C': [1, 10, 100], 'kernel': ['linear', 'rbf']}, cv=5, scoring='accuracy'); grid.fit(X, y); best_params_ = grid.best_params_.


In [ ]:
question = """
Pandas 데이터프레임(df)에 문자열 범주형 특성(object dtype)과 수치형 특성(float64 dtype)이 혼합되어 있으며, 수치형 특성에는 결측치(NaN)가 포함된 이진 분류 문제를 해결해야 합니다.

Scikit-learn의 모범 사례에 따라 Pipeline과 ColumnTransformer를 사용하여 이 데이터를 전처리하고, 이어서 RandomForestClassifier의 최적 하이퍼파라미터를 찾고자 합니다.

아래 요구사항에 따라 응답해 주십시오.

1. **전처리 파이프라인 구성:** ColumnTransformer 내에서 범주형 특성(categorical_features)과 수치형 특성(numerical_features) 각각에 적용해야 할 가장 적절한 Scikit-learn 변환기(Transformer) 이름을 제시하고, 그 변환기를 선택한 이유를 간결하게 설명하십시오.
2. **하이퍼파라미터 튜닝 전략:** 구축한 파이프라인의 최종 추정기(RandomForestClassifier)에 대해 GridSearchCV를 사용하여 탐색할 핵심 하이퍼파라미터 2가지를 제시하고, 파이프라인 내에서 해당 하이퍼파라미터를 지정할 때 사용해야 하는 정확한 매개변수 명명법(파이프라인_단계_이름__하이퍼파라미터)의 예시를 제시하십시오.
"""

print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: 
Pandas 데이터프레임(df)에 문자열 범주형 특성(object dtype)과 수치형 특성(float64 dtype)이 혼합되어 있으며, 수치형 특성에는 결측치(NaN)가 포함된 이진 분류 문제를 해결해야 합니다.

Scikit-learn의 모범 사례에 따라 Pipeline과 ColumnTransformer를 사용하여 이 데이터를 전처리하고, 이어서 RandomForestClassifier의 최적 하이퍼파라미터를 찾고자 합니다.

아래 요구사항에 따라 응답해 주십시오.

1. **전처리 파이프라인 구성:** ColumnTransformer 내에서 범주형 특성(categorical_features)과 수치형 특성(numerical_features) 각각에 적용해야 할 가장 적절한 Scikit-learn 변환기(Transformer) 이름을 제시하고, 그 변환기를 선택한 이유를 간결하게 설명하십시오.
2. **하이퍼파라미터 튜닝 전략:** 구축한 파이프라인의 최종 추정기(RandomForestClassifier)에 대해 GridSearchCV를 사용하여 탐색할 핵심 하이퍼파라미터 2가지를 제시하고, 파이프라인 내에서 해당 하이퍼파라미터를 지정할 때 사용해야 하는 정확한 매개변수 명명법(파이프라인_단계_이름__하이퍼파라미터)의 예시를 제시하십시오.


[답변]: 1) ColumnTransformer의 cat 트랜스포머에는 SimpleImputer(strategy='most_frequent')와 OneHotEncoder(handle_unknown='ignore')를 차례로 적용하고, numerical 트랜스포머에는 SimpleImputer(strategy='median')를 적용하는 파이프라인을 매핑하는 것이 적합합니다. 2) 최적 하이퍼파라미터를 찾기 위해 RandomForestClassifier의 n_estimators와 max_depth를 GridSearchCV로 탐색하는 것이 일반적입니다. 3) 파이프라인 내 매개변수명은 'rf__n_estim

In [ ]:
question = "Pandas DataFrame에서 결측값을 처리하고, Scikit-learn Pipeline으로 StandardScaler와 LogisticRegression을 적용한 후, Stratified K-Fold 교차검증까지 수행하는 전체 예제 코드를 보여줘"

print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas DataFrame에서 결측값을 처리하고, Scikit-learn Pipeline으로 StandardScaler와 LogisticRegression을 적용한 후, Stratified K-Fold 교차검증까지 수행하는 전체 예제 코드를 보여줘

[답변]: 다음은 pandas DataFrame에서 결측치를 처리하고, Pipeline으로 StandardScaler와 LogisticRegression를 적용한 뒤 StratifiedKFold 교차검증을 수행하는 전체 예제 코드입니다.

```python
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 샘플 데이터 생성 및 결측값 주입
np.random.seed(0)
df = pd.DataFrame({
    'feature1': np.random.randn(200),
    'feature2': np.random.randn(200),
    'feature3': np.random.randn(200),
})
for col in df.columns:
    df.loc[df.sample(frac=0.1).index, col] = np.nan

# 타깃 생성
df['target'] = (df['feature1'] + df['feature2'] > 0).astype(int)

X = df.drop(columns=['target'])
y = df['target']

# 파이프라인 구성: 결측값 임퓨테이션 > 표준화 > 로지스틱 회귀
pipe

## Retriever 세분화

In [71]:
# 전제: 현재 메모리에 로드된 통합 DB 변수: vectorstore
# 전제: 각 문서의 메타데이터에 {'library': 'pandas'} 또는 {'library': 'sklearn'}이 포함되어 있음

# 검색할 문서의 개수 (k)를 5개로 통일합니다.
SEARCH_KWARGS = {"k": 5}

# -----------------------------------------------------------
# 1. Pandas 전용 리트리버
# -----------------------------------------------------------
pandas_retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": SEARCH_KWARGS["k"],
        "filter": {"library": "pandas"}  # 메타데이터 'library'가 'pandas'인 문서만 필터링
    }
)
print("✅ Pandas 전용 리트리버 정의 완료.")

# -----------------------------------------------------------
# 2. Scikit-learn 전용 리트리버
# -----------------------------------------------------------
sklearn_retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": SEARCH_KWARGS["k"],
        "filter": {"library": "sklearn"} # 메타데이터 'library'가 'sklearn'인 문서만 필터링
    }
)
print("✅ Scikit-learn 전용 리트리버 정의 완료.")

# -----------------------------------------------------------
# 3. 둘 다 (통합) 리트리버
# -----------------------------------------------------------
# 필터링을 적용하지 않으면 자동으로 전체 DB에서 검색합니다.
combined_retriever = vectorstore.as_retriever(
    search_kwargs={"k": SEARCH_KWARGS["k"]} 
)
print("✅ 통합 (둘 다) 리트리버 정의 완료.")

✅ Pandas 전용 리트리버 정의 완료.
✅ Scikit-learn 전용 리트리버 정의 완료.
✅ 통합 (둘 다) 리트리버 정의 완료.


In [72]:
# 사용자 선택 값에 따라 리트리버를 반환하는 함수 (UI 연동 시 사용)
def select_retriever(user_selection: str):
    """
    사용자의 UI 선택 값에 따라 적절한 리트리버를 반환합니다.
    user_selection: 'pandas', 'sklearn', 또는 'combined'
    """
    if user_selection == 'pandas':
        return pandas_retriever
    elif user_selection == 'sklearn':
        return sklearn_retriever
    elif user_selection == 'combined':
        return combined_retriever
    else:
        # 기본값으로 통합 리트리버 사용
        return combined_retriever

# -----------------------------------------------------------
# 최종 RAG Chain 구성
# -----------------------------------------------------------
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# *RAG Chain은 이제 동적으로 선택된 리트리버를 사용합니다.*
# Chain의 구조는 그대로 유지됩니다.

# 예시: 사용자가 'Scikit-learn'을 선택했을 경우
current_retriever = select_retriever('sklearn')

rag_chain_dynamic = (
    {"context": current_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt # 이전에 수정한 균형 잡힌 프롬프트 사용
    | llm
    | StrOutputParser()
)

### 질의 테스트

In [73]:
question = "파이프라인 내에서 범주형 특성(Categorical Features)을 전처리하는 가장 좋은 방법을 코드로 보여주세요."
print(f"\n[질문]: {question}")

# --- 모드 1: Scikit-learn 전용 검색 ('sklearn') ---
user_selection = 'sklearn'
current_retriever = select_retriever(user_selection)
rag_chain_dynamic = (
    {"context": current_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)
response_sklearn = rag_chain_dynamic.invoke(question)
print(f"\n--- 모드 1: {user_selection} (필터링 성공) ---")
print(f"[답변]: {response_sklearn[:300]}...") # 답변의 앞부분만 출력하여 비교

# --- 모드 2: Pandas 전용 검색 ('pandas') ---
user_selection = 'pandas'
current_retriever = select_retriever(user_selection)
rag_chain_dynamic = (
    {"context": current_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)
response_pandas = rag_chain_dynamic.invoke(question)
print(f"\n--- 모드 2: {user_selection} (필터링 실패 예상) ---")
print(f"[답변]: {response_pandas[:300]}...") # Scikit-learn 문서를 찾지 못할 가능성이 높음


[질문]: 파이프라인 내에서 범주형 특성(Categorical Features)을 전처리하는 가장 좋은 방법을 코드로 보여주세요.

--- 모드 1: sklearn (필터링 성공) ---
[답변]: 범주형 특성은 파이프라인에서 ColumnTransformer를 사용해 한 곳에서 일관되게 인코딩하는 것이 가장 일반적이고 권장되는 방법입니다. 보통 OneHotEncoder를 활용하고, handle_unknown='ignore'로 테스트 데이터에서 unseen 카테고리에 안전하게 대응하며, 희소 행렬로 메모리 사용을 줄이는 것이 좋습니다. 모델 유형에 따라서는 고카디널리티(고유 값이 많은) 특성은 Target Encoding 등 다른 인코딩 방법을 고려할 수 있습니다. 아래 예시는 일반적인 케이스와 선택적으로 High-Cardina...

--- 모드 2: pandas (필터링 실패 예상) ---
[답변]: 파이프라인에서 범주형 특성을 처리할 때는 보통 scikit-learn의 ColumnTransformer와 OneHotEncoder를 함께 사용합니다. 이렇게 하면 수치형과 범주형 특성을 같은 파이프라인에서 다루면서, 학습 데이터의 카테고리 구성을 기준으로 일관되게 인코딩하고, 테스트 데이터에 보지 못한 카테고리가 있어도 handle_unknown='ignore'로 안전하게 처리할 수 있습니다. 필요 시 학습 데이터에서 사용할 전체 범주를 미리 고정해 일관성을 더 강화할 수 있습니다.

다음은 예시 코드입니다. 간단한 이진 분류 데이...
